In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf 

from baseline import Baseline
from system_t import System_T

# Load Mixed-MNIST dataset from keras

In [2]:
from extra_keras_datasets import emnist

(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = emnist.load_data(type='balanced')
mnist_train_images = mnist_train_images.reshape((-1, 28, 28, 1)) / 255.0
mnist_test_images = mnist_test_images.reshape((-1, 28, 28, 1)) / 255.0

idx = mnist_train_labels <= 9
mnist_train_images = mnist_train_images[idx]
mnist_train_labels = mnist_train_labels[idx]

fashion = tf.keras.datasets.fashion_mnist
(fashion_train_images,fashion_train_labels), (fashion_test_images, fashion_test_labels) = fashion.load_data()

fashion_train_labels = fashion_train_labels + 10
fashion_test_labels = fashion_test_labels + 10
fashion_train_images = fashion_train_images.reshape((60000, 28, 28, 1)) / 255.0
fashion_test_images = fashion_test_images.reshape((10000, 28, 28, 1)) / 255.0

train_images = np.concatenate((mnist_train_images, fashion_train_images), axis=0)
train_labels = np.concatenate((mnist_train_labels, fashion_train_labels), axis=0)

# Basic setting: slices have the same amounts of data

In [3]:
def shuffle(data, label):
    shuffle = np.arange(len(data))
    np.random.shuffle(shuffle)
    data = data[shuffle]
    label = label[shuffle]
    return data, label

num_class = len(np.unique(train_labels))
print("Number of slices : %d" % num_class)

y_train_one_hot = tf.keras.utils.to_categorical(train_labels)
mixed_data = (train_images, y_train_one_hot)

initial_data_array = []
val_data_dict = []
add_data_dict = []
    
val_data_num = 500

for i in range(num_class):
    data_num = 150
    initial_data_array.append(data_num)
    idx = np.argmax(mixed_data[1], axis=1) == i

    val_data_dict.append((mixed_data[0][idx][data_num:data_num+val_data_num], mixed_data[1][idx][data_num:data_num+val_data_num]))
    add_data_dict.append((mixed_data[0][idx][data_num+val_data_num:], mixed_data[1][idx][data_num+val_data_num:]))

    if i == 0:
        train_data = mixed_data[0][idx][:data_num]
        train_label = mixed_data[1][idx][:data_num]
        val_data = mixed_data[0][idx][data_num:data_num+val_data_num]
        val_label = mixed_data[1][idx][data_num:data_num+val_data_num]
    else:
        train_data = np.concatenate((train_data, mixed_data[0][idx][:data_num]), axis=0)
        train_label = np.concatenate((train_label, mixed_data[1][idx][:data_num]), axis=0) 
        val_data = np.concatenate((val_data, mixed_data[0][idx][data_num:data_num+val_data_num]), axis=0)
        val_label = np.concatenate((val_label, mixed_data[1][idx][data_num:data_num+val_data_num]), axis=0)   
    
train_data, train_label = shuffle(train_data, train_label)
print(train_data.shape, train_label.shape, num_class)

Number of slices : 20
(3000, 28, 28, 1) (3000, 20) 20


# Define slices

In [4]:
slice_desc = []
# Mixed mnist
a = ["Digit 0", "Digit 1","Digit 2","Digit 3","Digit 4","Digit 5","Digit 6","Digit 7","Digit 8","Digit 9",
   "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

for i in range(num_class):
    slice_desc.append('Slice: %s' % (a[i]))
    print('Slice: %s, Initial size: %s' % (a[i], initial_data_array[i]))

Slice: Digit 0, Initial size: 150
Slice: Digit 1, Initial size: 150
Slice: Digit 2, Initial size: 150
Slice: Digit 3, Initial size: 150
Slice: Digit 4, Initial size: 150
Slice: Digit 5, Initial size: 150
Slice: Digit 6, Initial size: 150
Slice: Digit 7, Initial size: 150
Slice: Digit 8, Initial size: 150
Slice: Digit 9, Initial size: 150
Slice: T-shirt/top, Initial size: 150
Slice: Trouser, Initial size: 150
Slice: Pullover, Initial size: 150
Slice: Dress, Initial size: 150
Slice: Coat, Initial size: 150
Slice: Sandal, Initial size: 150
Slice: Shirt, Initial size: 150
Slice: Sneaker, Initial size: 150
Slice: Bag, Initial size: 150
Slice: Ankle boot, Initial size: 150


# Original ( with no data acquisition ) 

In [5]:
cost_func = [1] * num_class
lr = 0.0005

ori = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
ori.performance(budget=0, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 0
======= Collect Data =======
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
======= Performance =======
Loss: 0.30089 (0.00644), Average EER: 0.20728 (0.00725), Max EER: 0.91888 (0.11251)



# System T Demo on Mixed-MNIST

## Use 3000 budget, lambda=0.1, "Moderate" strategy

In [6]:
budget = 3000
method = 'Moderate'

st = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, add_data_dict)
st.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=2000, cost_func=cost_func, 
                 Lambda=0.1, num_iter=5, slice_desc=slice_desc, strategy=method, show_figure=False)

======= Collect Data =======
[  5   9  50  25  41  29   8  14  51  43  32  11 113  81 146  31 150  79
   0   0]
Total Cost: 918, Remaining Budget: 2082
======= Collect Data =======
[ 11   0  31  46  47  53   0  71 122  72  36  16 273 172 170  29 300   0
  44   0]
Total Cost: 1493, Remaining Budget: 589
======= Collect Data =======
[  5   0   0   0   0   0   0   0   0   0 341  14   0   0   8   6  71  38
  85  21]
Total Cost: 589, Remaining Budget: 0

======= Performance =======
[ 21.   9.  81.  71.  88.  82.   8.  85. 173. 115. 409.  41. 386. 253.
 324.  66. 521. 117. 129.  21.]
Number of iteration: 3
Strategy: Moderate, C: 0.1, Budget: 3000
Loss: 0.23512 (0.00293), Average EER: 0.14696 (0.00485), Max EER: 0.48164 (0.04695)



# Baseline: Uniform ( = Water filling )

## For a basic setting, Uniform method is equivalent to Water filling method

In [7]:
budget = 3000
uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=10, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 3000
======= Collect Data =======
[150 150 150 150 150 150 150 150 150 150 150 150 150 150 150 150 150 150
 150 150]
======= Performance =======
Loss: 0.24014 (0.00339), Average EER: 0.18189 (0.00475), Max EER: 0.67253 (0.08671)



# Summary of results

| Method | Loss | Avg.EER |
|:---------------------:|:---------------------:|:---------------------:|
| Original | 0.30089 (± 0.00322) | 0.20728 (± 0.00363) |
| Uniform | 0.24014 (± 0.00169) | 0.18189 (± 0.00237) |
| Water filling | 0.24014 (± 0.00169) | 0.18189 (± 0.00237) |
| Moderate (ours) | 0.23512 (± 0.00146) | 0.14696 (± 0.00243) |